In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [ ]:
(Xtrain, ytrain), (Xtest,ytest) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
# reshape and normalize
Xtrain = Xtrain.reshape(-1,28,28,1).astype("float32") / 255.0
Xtest = Xtest.reshape(-1,28,28,1).astype("float32") / 255.0

In [ ]:
model = keras.Sequential([
            layers.Input(shape=(28,28,1)),
            layers.Conv2D(64,3,padding='same'),
            layers.ReLU(),
            layers.Conv2D(128,3,padding='same'),
            layers.ReLU(),
            layers.Flatten(),
            layers.Dense(10)

],name='model')

Custom Training 

In [ ]:
class CustomFit(keras.Model):
  def __init__(self,model):
    super(CustomFit,self).__init__()
    self.model = model
  
  def train_step(self,data):
    x,y = data
    #forward propagation
    with tf.GradientTape() as tape:
      y_pred = self.model(x,training=True)
      loss = self.compiled_loss(y,y_pred)

    #Getting the gradients
    training_vars = self.trainable_variables
    gradient = tape.gradient(loss,training_vars) # gradient of loss w.r.t training variables

    #optimizer step
    self.optimizer.apply_gradients(zip(gradient,training_vars))

    #accuracy part
    self.compiled_metrics.update_state(y,y_pred)

    #returning loss and accuracy metrics
    return {m.name : m.result() for m in self.metrics}




In [ ]:
training = CustomFit(model)
training.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

In [ ]:
training.fit(Xtrain,ytrain,batch_size=128,epochs=1)

469/469 [==============================] - 338s 719ms/step - loss: 0.0978 - accuracy: 0.9710


Custom optimizer, loss and testing step.

In [ ]:
class CustomFit1(keras.Model):
  def __init__(self,model):
    super(CustomFit1,self).__init__()
    self.model = model

 # Compiling the model 
  def compile(self,optimizer,loss):
    super(CustomFit1,self).compile()
    self.optimizer = optimizer
    self.loss = loss

  #Testing the model
  def test_step(self,data):
    x,y = data

    y_pred = self.model(x,training=False)
    loss = self.loss(y,y_pred)
    acc_metrics.update_state(y,y_pred)

    return {"loss":loss,"accuracy":acc_metrics.result()}

  def train_step(self,data):
    x,y = data

    #forward propagation
    with tf.GradientTape() as tape:
      y_pred = self.model(x,training=True)
      loss = self.loss(y,y_pred)

    #getting the gradient
    training_vars = self.trainable_variables
    gradients = tape.gradient(loss,training_vars)

    #applying gradients optimizer 
    self.optimizer.apply_gradients(zip(gradients,training_vars))

    #calc accuracy
    acc_metrics.update_state(y,y_pred)

    return {"loss":loss, "accuracy":acc_metrics.result()}

In [ ]:
acc_metrics = keras.metrics.SparseCategoricalAccuracy(name='accuracy')

In [ ]:
model = keras.Sequential([
            layers.Input(shape=(28,28,1)),
            layers.Conv2D(64,3,padding='same'),
            layers.ReLU(),
            layers.Conv2D(128,3,padding='same'),
            layers.ReLU(),
            layers.Flatten(),
            layers.Dense(10)

],name='model')

In [ ]:
training1 = CustomFit1(model)
training1.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [ ]:
training1.fit(Xtrain,ytrain,batch_size=128,epochs=1)

469/469 [==============================] - 338s 720ms/step - loss: 0.1442 - accuracy: 0.9045


In [ ]:
training1.evaluate(Xtest,ytest,batch_size=128)

79/79 [==============================] - 16s 197ms/step - loss: 0.0462 - accuracy: 0.9589


[0.9609571695327759, 0.0006942173349671066]